## Qs Construction

In [1]:
import kProcessor as kp
import random
import hashlib
import json

In [2]:
# Mimic the function in kProcessor of coversion from str_to_int
def str_to_int(kmer):
    _map = {"A":0, "C":1, "T":2, "G":3}
    strint = 0
    for n in kmer:
        curr = _map[n]
        strint = strint | curr
        strint = strint << 2
        
    return strint >> 2

def int_to_str(kmer, kSize):
    _map = {0: 'A', 1: 'C', 2: 'T', 3: 'G'}
    kmer_str = ""
    for i in range(kSize, 0, -1):
        base = (kmer >> (i*2-2)) & 3
        ch = _map[base]
        kmer_str += ch
    
    return kmer_str

# Function to match 2 kmers with common first Q*2 bits or First Q Chars
# Return bool value, True if match, False if not
def matchingKmers(kmer1_int, kmer2_int, kSize, Q):
    msk = (~(-1<<Q*2)) << (kSize*2 - Q*2)
    return ((kmer1_int ^ kmer2_int) & msk == 0)

def mask(kSize, Q):
    return (~(-1<<Q*2)) << (kSize*2 - Q*2)

# Take list of colors, sort it, and create a return a hash value
def create_super_color(colors):
    val = hashlib.md5(str(sorted(list(set(colors)))).encode()).hexdigest()[:9]
    print("Hashing: %s to %s" % (str(sorted(colors)) , val) )
    return val

## Index Loading

In [3]:
## Double loading will be changed in the future after implementing coloredkDataFrame iterator
kf = kp.kDataFrame.load("idx_min_test")
#ckf = kp.colored_kDataFrame.load("idx_min_test") # No need to load it as colored, we don't need samples IDs information
kSize = kf.getkSize()

### TEST kFrame, Only first 14 kmers

In [69]:
no_of_kmers = 15
test_kf = kp.kDataFrameMQF(kSize)
it = kf.begin()
for i in range(no_of_kmers):
    test_kf.insert(it.getKmer(), it.getKmerCount())
    it.next()

print("Inserted %d kmers" % test_kf.size())

Inserted 9 kmers


In [70]:
kk = test_kf.begin()
all_kk = []
while(kk != test_kf.end()):
    all_kk.append(kk.getKmer())
    kk.next()

# Determine minQ and maxQ and get list of masks & superColorsDIct initialization

In [159]:
minQ = 1
stepQ = 1
maxQ = 4
superColors = {}
temp_superColors = {}
superColorsCount = {}
masks = {}

# for Q in range(maxQ, minQ-1, -stepQ):
for Q in range(minQ, maxQ + 1, stepQ):
    masks[Q] = mask(kSize, Q)
    superColors[Q] = {}
    superColorsCount[Q] = {}
    temp_superColors[Q] = []

#print(temp_superColors)
#print(superColors)
#print(superColorsCount)

list(map(bin,masks.values()))

['0b110000000000000000000000000000',
 '0b111100000000000000000000000000',
 '0b111111000000000000000000000000',
 '0b111111110000000000000000000000']

In [160]:
all_kmers = set()
it = kf.begin()
prev_kmer = it.getHashedKmer()
prev_kmer_color = it.getKmerCount()
all_kmers.add(int_to_str(prev_kmer, kSize))
count = 15

# Create list of kmers
while it != kf.end():
    
    if count == 0:
        break
    count -= 1
    
    it.next()
    curr_kmer = it.getHashedKmer()
    curr_kmer_color = it.getKmerCount()
    
    # Apply XOR to kmer1 and kmer2 (single time per iteration)
    xor = prev_kmer ^ curr_kmer
    
    # Apply all masks with all Qs
    for Q, MASK in masks.items():
        
        matched = not bool(xor & MASK) # True if there's match, False if not
        
        if matched:
            print("Matching Q%d %s & %s | TRUE | [prevC:%d, currC=%d]" % (Q, int_to_str(prev_kmer, kSize),int_to_str(curr_kmer, kSize), prev_kmer_color, curr_kmer_color))
            temp_superColors[Q] += [prev_kmer_color, curr_kmer_color]
            
        else:
            print("Matching Q%d %s & %s | FALSE | [prevC:%d, currC=%d]" % (Q, int_to_str(prev_kmer, kSize),int_to_str(curr_kmer, kSize),  prev_kmer_color, curr_kmer_color))
            temp_superColors[Q].append(prev_kmer_color)
            super_color_id = create_super_color(temp_superColors[Q])
            
            # Check if the superColor already exist
            # If yes: increment the count to one
            # If No:  Insert the new superColor and set the count to 1
            if super_color_id not in superColors[Q]:
                superColors[Q][super_color_id] = list(set(temp_superColors[Q]))
                superColorsCount[Q][super_color_id] = 1

            else:
                # IF the supercolor already exist, just increment it
                superColorsCount[Q][super_color_id] += 1
                
            temp_superColors[Q] = [curr_kmer_color]
    
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
#     for Q, colors in temp_superColors.items():
#         super_color_id = create_super_color(colors)
        
#         if super_color_id not in superColors[Q]:
#             superColors[Q][super_color_id] = list(set(colors))
#             superColorsCount[Q][super_color_id] = 1
#             temp_superColors[Q] = []

#         else:
#             superColorsCount[Q][super_color_id] += 1
#             temp_superColors[Q] = []

    #print("---------------------------------------")
    
    all_kmers.add(int_to_str(prev_kmer, kSize)) ## ADDITIONAL
    all_kmers.add(int_to_str(curr_kmer, kSize)) ## ADDITIONAL
    prev_kmer = curr_kmer
    prev_kmer_color = curr_kmer_color
    
# If the last iteration got a match, push it to the superColors
if matched:
    for Q, colors in temp_superColors.items():
        colors.remove(curr_kmer_color)
        super_color_id = create_super_color(colors)

        if super_color_id not in superColors[Q]:
            
            superColors[Q][super_color_id] = list(set(colors))
            superColorsCount[Q][super_color_id] = 1

        else:
            # IF the supercolor already exist, just increment it
            superColorsCount[Q][super_color_id] += 1



Matching Q1 AAAATTTCCAGTACA & AAAATTTTAAAAGAA | TRUE | [prevC:27, currC=55]
Matching Q2 AAAATTTCCAGTACA & AAAATTTTAAAAGAA | TRUE | [prevC:27, currC=55]
Matching Q3 AAAATTTCCAGTACA & AAAATTTTAAAAGAA | TRUE | [prevC:27, currC=55]
Matching Q4 AAAATTTCCAGTACA & AAAATTTTAAAAGAA | TRUE | [prevC:27, currC=55]
+++++++++++++++++++++++++++++++++++++++++++++++
Matching Q1 AAAATTTTAAAAGAA & AAAATTTTTTGTTTT | TRUE | [prevC:55, currC=18]
Matching Q2 AAAATTTTAAAAGAA & AAAATTTTTTGTTTT | TRUE | [prevC:55, currC=18]
Matching Q3 AAAATTTTAAAAGAA & AAAATTTTTTGTTTT | TRUE | [prevC:55, currC=18]
Matching Q4 AAAATTTTAAAAGAA & AAAATTTTTTGTTTT | TRUE | [prevC:55, currC=18]
+++++++++++++++++++++++++++++++++++++++++++++++
Matching Q1 AAAATTTTTTGTTTT & AAAATTTGTTCCTTT | TRUE | [prevC:18, currC=18]
Matching Q2 AAAATTTTTTGTTTT & AAAATTTGTTCCTTT | TRUE | [prevC:18, currC=18]
Matching Q3 AAAATTTTTTGTTTT & AAAATTTGTTCCTTT | TRUE | [prevC:18, currC=18]
Matching Q4 AAAATTTTTTGTTTT & AAAATTTGTTCCTTT | TRUE | [prevC:18, cu

In [161]:
print("SUPER COLORS")
print(json.dumps(superColors, sort_keys=True, indent=4, separators=(',', ': ')))
print("\n--------------------------------\n\n")
print("SUPER COLORS COUNT")
print(json.dumps(superColorsCount, sort_keys=True, indent=4, separators=(',', ': ')))
print("\n--------------------------------\n\n")
print("TEMP SUPER COLORS")
print(json.dumps(temp_superColors, sort_keys=True, indent=4, separators=(',', ': ')))
print("\n--------------------------------\n\n")

total_sum = {}
for Q, counts in superColorsCount.items():
    total_sum[Q] = 0
    for hash_value, count in counts.items():
        total_sum[Q] += len(superColors[Q][hash_value]) * count

print("TOTAL NUMBER OF KMERS:")
print(json.dumps(total_sum, sort_keys=True, indent=4, separators=(',', ': ')))
print("\n--------------------------------\n\n")


SUPER COLORS
{
    "1": {
        "928afc491": [
            32,
            1,
            18,
            55,
            25,
            27
        ]
    },
    "2": {
        "928afc491": [
            32,
            1,
            18,
            55,
            25,
            27
        ]
    },
    "3": {
        "928afc491": [
            32,
            1,
            18,
            55,
            25,
            27
        ]
    },
    "4": {
        "928afc491": [
            32,
            1,
            18,
            55,
            25,
            27
        ]
    }
}

--------------------------------


SUPER COLORS COUNT
{
    "1": {
        "928afc491": 1
    },
    "2": {
        "928afc491": 1
    },
    "3": {
        "928afc491": 1
    },
    "4": {
        "928afc491": 1
    }
}

--------------------------------


TEMP SUPER COLORS
{
    "1": [
        27,
        55,
        55,
        18,
        18,
        18,
        18,
        1,
        1,
        1

In [150]:
create_super_color([55,1])

Hashing: [1, 55] to b36a335ff


'b36a335ff'

In [ ]:
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))